In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from multiclass_metrics import ClassificationMetrics
from sklearn import datasets
from sklearn.model_selection import train_test_split
from models_2 import DecisionTree, RandomForest
sys.path.append(os.path.abspath('../..'))
from data_splitting import train_val_split

df = pd.read_csv('/home/facuvulcano/Machine-Learning/Vulcano_Facundo_TP3/Problema_2/data/raw/diabetes_dev.csv')
df_test = pd.read_csv('/home/facuvulcano/Machine-Learning/Vulcano_Facundo_TP3/Problema_2/data/raw/diabetes_test.csv')

In [2]:
X_train, X_val, y_train, y_val = train_val_split(df, 'Diabetes')

## **LDA**

https://www.evidentlyai.com/classification-metrics

## **Multiclass Logistic Regression**

## **Random Forest**

In [3]:
clf = RandomForest()
clf.fit(X_train.values, y_train.astype(int).values)
predictions = clf.predict(X_val.values)


In [4]:
metrics = ClassificationMetrics(y_val.values, predictions)